In [3]:
import pandas as pd
import numpy as np
import csv

Read in data

In [31]:
datPestEst = pd.read_csv('data/EPest_county_estimates_2013_2017_v2.csv')
datPest2 = pd.read_csv('data/combined_pest.csv')

In [33]:
#combining the for cleaning
datPest = pd.concat([datPest2,datPestEst])

In [35]:
datPest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8148801 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   COMPOUND          object 
 1   YEAR              int64  
 2   STATE_FIPS_CODE   int64  
 3   COUNTY_FIPS_CODE  int64  
 4   EPEST_LOW_KG      float64
 5   EPEST_HIGH_KG     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 435.2+ MB


In [37]:
datPest.head()

,COMPOUND,YEAR,STATE_FIPS_CODE,COUNTY_FIPS_CODE,EPEST_LOW_KG,EPEST_HIGH_KG
0,"2,4-D",1992,1,1,233.3,913.8
1,"2,4-D",1992,1,3,2895.2,3821.3
2,"2,4-D",1992,1,5,1114.7,1836.2
3,"2,4-D",1992,1,7,404.3,882.0
4,"2,4-D",1992,1,9,1276.5,1568.2


In [39]:
datPest['STATE_FIPS_CODE'] = datPest['STATE_FIPS_CODE'].astype(str)
datPest['COUNTY_FIPS_CODE'] = datPest['COUNTY_FIPS_CODE'].astype(str)
datPest['FIPS'] = datPest['STATE_FIPS_CODE'].str.cat(datPest['COUNTY_FIPS_CODE'], sep = '')
datPest = datPest.drop(columns=['STATE_FIPS_CODE','COUNTY_FIPS_CODE'])

In [40]:
datPestP = datPest.pivot_table(
    index=["YEAR", "FIPS"], 
    columns="COMPOUND", 
    values=["EPEST_LOW_KG", "EPEST_HIGH_KG"]
)
datPestP = datPestP.reset_index()

In [43]:
#replace NaN with 0 
#if no data, we will assume that the level was 0 for this modeling
datPestP = datPestP.fillna(0)

In [45]:
# Flatten the MultiIndex columns
datPestP.columns = ['_'.join(col).strip() for col in datPestP.columns.values]
# Rename the first two columns 
datPestP = datPestP.rename(columns={'YEAR_': 'YEAR', 'FIPS_': 'FIPS'})

In [47]:
datPestP.head()

,YEAR,FIPS,EPEST_HIGH_KG_1-METHYL CYCLOPROPENE,"EPEST_HIGH_KG_2,4-D","EPEST_HIGH_KG_2,4-DB",EPEST_HIGH_KG_6-BENZYLADENINE,EPEST_HIGH_KG_ABAMECTIN,EPEST_HIGH_KG_ABSCISIC ACID,EPEST_HIGH_KG_ACEPHATE,EPEST_HIGH_KG_ACEQUINOCYL,...,EPEST_LOW_KG_TRINEXAPAC,EPEST_LOW_KG_TRITICONAZOLE,EPEST_LOW_KG_UNICONAZOLE,EPEST_LOW_KG_VERNOLATE,EPEST_LOW_KG_VINCLOZOLIN,EPEST_LOW_KG_ZETA-CYPERMETHRIN,EPEST_LOW_KG_ZINC,EPEST_LOW_KG_ZINEB,EPEST_LOW_KG_ZIRAM,EPEST_LOW_KG_ZOXAMIDE
0,1992,101,0.0,10134.2,437.9,0.0,0.0,0.0,348.8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1992,103,0.0,2812.9,720.2,0.0,0.0,0.0,209.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.6,0.0
2,1992,105,0.0,22793.0,578.9,0.0,0.0,0.0,763.3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1992,11,0.0,913.8,31.4,0.0,0.0,0.0,188.8,0.0,...,0.0,0.0,0.0,16.8,1.1,9.6,0.0,0.0,0.0,0.0
4,1992,1101,0.0,1364.5,73.6,0.0,0.0,0.0,289.4,0.0,...,0.0,0.0,0.0,2.1,0.0,1.4,0.0,0.0,0.0,0.0


In [49]:
datPest_filtered = datPestP[datPestP['YEAR'] >2009]

In [51]:
datPest_filtered.head()

,YEAR,FIPS,EPEST_HIGH_KG_1-METHYL CYCLOPROPENE,"EPEST_HIGH_KG_2,4-D","EPEST_HIGH_KG_2,4-DB",EPEST_HIGH_KG_6-BENZYLADENINE,EPEST_HIGH_KG_ABAMECTIN,EPEST_HIGH_KG_ABSCISIC ACID,EPEST_HIGH_KG_ACEPHATE,EPEST_HIGH_KG_ACEQUINOCYL,...,EPEST_LOW_KG_TRINEXAPAC,EPEST_LOW_KG_TRITICONAZOLE,EPEST_LOW_KG_UNICONAZOLE,EPEST_LOW_KG_VERNOLATE,EPEST_LOW_KG_VINCLOZOLIN,EPEST_LOW_KG_ZETA-CYPERMETHRIN,EPEST_LOW_KG_ZINC,EPEST_LOW_KG_ZINEB,EPEST_LOW_KG_ZIRAM,EPEST_LOW_KG_ZOXAMIDE
53519,2010,101,0.0,18886.5,32.2,0.0,103.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53520,2010,103,0.0,3609.3,13.3,0.0,43.2,0.0,465.7,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.9,0.0
53521,2010,105,0.0,13599.5,43.8,0.0,187.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.7
53522,2010,11,0.0,2490.3,45.2,0.0,26.3,0.0,532.3,0.0,...,0.0,0.0,0.0,0.0,0.0,7.4,0.0,0.0,0.0,0.0
53523,2010,1101,0.0,5577.5,0.9,0.0,8.9,0.0,194.8,0.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0


In [53]:
#datPestP.to_csv("pesticide_prediction_data.csv")